### Priemer League Standings ETL Pipeline 

#####     This Notebook extracts Priemer League standings from Football Data Api ,transforms the data and loads it into MySQL data base  

### 1.  Install Required Packages 

In [32]:
# !pip install requests pandas mysql-connector-python python-env

import requests 
import pandas as pd 
import mysql.connector 
from datetime import datetime
from dotenv import load_dotenv
import os


In [23]:
def extract_data(api_key):
    url = "https://api.football-data.org/v4/competitions/PL/standings"
    headers = {"X-Auth-Token": api_key}

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

### 2 . Load Enviromrnt Variables  

In [35]:
load_dotenv()

API_KEY = os.getenv("API_KEY")
API_URL = "https://api.football-data.org/v4/competitions/PL/standings"

DB_CONFIG = {
    "host": os.getenv("DB_HOST", "localhost"),
    "user": os.getenv("DB_USER", "root"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME", "football")
}

print("✅ Environment variables loaded successfully")
print(f"📡 API URL: {API_URL}")
print(f"🗄️  Database: {DB_CONFIG['database']}")

✅ Environment variables loaded successfully
📡 API URL: https://api.football-data.org/v4/competitions/PL/standings
🗄️  Database: football


### 4 . Extract - Get data From API

In [36]:
def extract_data():
    """Get data from Football Data API"""
    print("Extracting data from API...")
    
    headers = {"X-Auth-Token": API_KEY}
    response = requests.get(API_URL, headers=headers, timeout=30)
    response.raise_for_status()
    
    data = response.json()
    print("✅ Data extracted successfully")
    return data

In [37]:
# Run extraction 
raw_data = extract_data()

Extracting data from API...
✅ Data extracted successfully


### TRANSFORM - Clean and Structure Data 

In [38]:
def transform_data(raw_data):
    """Transform JSON data to DataFrame"""
    print("🔄 Transforming data...")
    
    # Extract standings table
    table = raw_data["standings"][0]["table"]
    
    # Extract season
    season_start = raw_data["season"]["startDate"][:4]
    season_end = raw_data["season"]["endDate"][:4]
    season = f"{season_start}-{season_end}"
    
    # Build rows
    rows = []
    for team in table:
        rows.append({
            "season": season,
            "position": team["position"],
            "team": team["team"]["name"],
            "played": team["playedGames"],
            "wins": team["won"],
            "draws": team["draw"],
            "losses": team["lost"],
            "goals_for": team["goalsFor"],
            "goals_against": team["goalsAgainst"],
            "goal_diff": team["goalDifference"],
            "points": team["points"]
        })
    
    df = pd.DataFrame(rows)
    print(f"✅ Transformed {len(df)} teams for season {season}")
    return df

In [39]:
# Run transformation
df_standings = transform_data(raw_data)

🔄 Transforming data...
✅ Transformed 20 teams for season 2025-2026


In [41]:
# Display the data
print("\n Premier League Standings:")
display(df_standings.head())


 Premier League Standings:


,season,position,team,played,wins,draws,losses,goals_for,goals_against,goal_diff,points
0,2025-2026,1,Arsenal FC,21,15,4,2,40,14,26,49
1,2025-2026,2,Manchester City FC,21,13,4,4,45,19,26,43
2,2025-2026,3,Aston Villa FC,21,13,4,4,33,24,9,43
3,2025-2026,4,Liverpool FC,21,10,5,6,32,28,4,35
4,2025-2026,5,Brentford FC,21,10,3,8,35,28,7,33


### 6 . Data Quality Checks 

In [42]:
# 🔍 Data Quality Report:

print(f"  • Total teams: {len(df_standings)}")
print(f"  • Season: {df_standings['season'].iloc[0]}")
print(f"  • Missing values: {df_standings.isnull().sum().sum()}")
print(f"  • Total points distributed: {df_standings['points'].sum()}")
print(f"  • Total goals scored: {df_standings['goals_for'].sum()}")

  • Total teams: 20
  • Season: 2025-2026
  • Missing values: 0
  • Total points distributed: 575
  • Total goals scored: 587


In [43]:
# Show top 5 teams
print("\n🏅 Top 5 Teams:")
print(df_standings[['position', 'team', 'points', 'goal_diff']].head())


🏅 Top 5 Teams:
   position                team  points  goal_diff
0         1          Arsenal FC      49         26
1         2  Manchester City FC      43         26
2         3      Aston Villa FC      43          9
3         4        Liverpool FC      35          4
4         5        Brentford FC      33          7


### 7. LOAD - Insert into Mysql DataBase 

In [44]:
def load_to_mysql(df):
    """Load data into MySQL database"""
    print(" Loading data to MySQL...")
    
    # Connect to database
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()
    
    # Get season for deletion
    season = df['season'].iloc[0]
    
    # Delete existing data for this season (incremental load)
    cursor.execute("DELETE FROM premier_league_standings WHERE season = %s", (season,))
    deleted_rows = cursor.rowcount
    print(f"🗑️  Deleted {deleted_rows} old records for season {season}")
    
    # Insert new data
    insert_query = """
    INSERT INTO premier_league_standings
    (season, position, team, played, wins, draws, losses,
     goals_for, goals_against, goal_diff, points)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    rows_inserted = 0
    for _, row in df.iterrows():
        cursor.execute(insert_query, tuple(row))
        rows_inserted += 1
    
    conn.commit()
    print(f"✅ Inserted {rows_inserted} rows successfully")
    
    # Cleanup
    cursor.close()
    conn.close()

In [45]:
# Run load
load_to_mysql(df_standings)

 Loading data to MySQL...
🗑️  Deleted 0 old records for season 2025-2026
✅ Inserted 20 rows successfully


### 8. Verify Data in DataBase 

In [46]:
def verify_load():

    conn = mysql.connector.connect(**DB_CONFIG)

    query = """
    SELECT season, position, team, played, wins, draws, losses, 
           goals_for, goals_against, goal_diff, points
    FROM premier_league_standings
    ORDER BY position
    LIMIT 10
    """
    df_verify = pd.read_sql(query, conn)
    conn.close()

    print(f"✅ Found {len(df_verify)} records in database")
    return df_verify

In [47]:
# Verify load
df_verified = verify_load()
print("\n Verified Data from MySQL:")
display(df_verified)

✅ Found 10 records in database

 Verified Data from MySQL:


C:\Users\h\AppData\Local\Temp\ipykernel_3416\3127006974.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_verify = pd.read_sql(query, conn)


,season,position,team,played,wins,draws,losses,goals_for,goals_against,goal_diff,points
0,2025-2026,1,Arsenal FC,21,15,4,2,40,14,26,49
1,2025-2026,2,Manchester City FC,21,13,4,4,45,19,26,43
2,2025-2026,3,Aston Villa FC,21,13,4,4,33,24,9,43
3,2025-2026,4,Liverpool FC,21,10,5,6,32,28,4,35
4,2025-2026,5,Brentford FC,21,10,3,8,35,28,7,33
5,2025-2026,6,Newcastle United FC,21,9,5,7,32,27,5,32
6,2025-2026,7,Manchester United FC,21,8,8,5,36,32,4,32
7,2025-2026,8,Chelsea FC,21,8,7,6,34,24,10,31
8,2025-2026,9,Fulham FC,21,9,4,8,30,30,0,31
9,2025-2026,10,Sunderland AFC,21,7,9,5,21,22,-1,30
